In [1]:
from pysat.formula import CNF, IDPool
from pysat.solvers import Minisat22
from pysat.card import CardEnc, EncType

In [2]:
def myvar(i):
    return vpool.id(f'myvar@{i}')

In [3]:
vpool = IDPool(start_from=1)
cnf = CNF()
N = 10
vs = [myvar(i) for i in range(1, N+1)]

# Propagate 1's to the right, i.e. the solution is "00..011..1"
for i in range(1, N):
    cnf.append((-myvar(i), myvar(i+1)))

cnf.append([-myvar(1)])
cnf.extend(CardEnc.atleast(vs, 2, vpool=vpool, encoding=EncType.totalizer).clauses)
cnf.extend(CardEnc.atmost(vs, 7, vpool=vpool, encoding=EncType.totalizer).clauses)

In [4]:
with Minisat22(bootstrap_with=cnf.clauses) as s:
    if s.solve(assumptions=vs[::-1]):
    #if s.solve():
        model = [None] + [x > 0 for x in s.get_model()]
        print("SAT")
        print(list(range(1, N+1)))
        print(f"{[int(model[v]) for v in vs]}")
    else:
        print("UNSAT")
        print(f"Core: {s.get_core()}")

UNSAT
Core: [3, 4, 5, 6, 7, 8, 9, 10]


In [16]:
#n = 5
#k = 5
#edges = [(1, 2), (2, 3), (3, 4), (4, 1), (1, 5), (2, 5), (3, 5), (4, 5)]
n = 4
k = 4
edges = [(1, 2), (2, 3), (1, 3), (1, 4), (2, 4), (3, 4)]

In [17]:
# static k

vpool = IDPool(start_from=1)
cnf = CNF()

for v in range(1, n + 1):
    for i in range(k):
        for j in range(i + 1, k):
            cnf.append([-myvar(v * k + i), -myvar(v * k + j)])
for v in range(1, n + 1):
    cnf.append([myvar(v * k + i) for i in range(k)])
for a, b in edges:
    for i in range(k):
        cnf.append([-myvar(a * k + i), -myvar(b * k + i)])
with Minisat22(bootstrap_with=cnf.clauses) as s:
    if s.solve():
        colors = {key // k : key % k for key, value in zip(list(range(k, (n + 1) * k)), s.get_model()) if value > 0}
        print(f"SAT {colors}")
    else:
        print("UNSAT")

SAT {1: 3, 2: 2, 3: 1, 4: 0}


In [18]:
# dinamic k

vpoll = IDPool(start_from=1)
cnf = CNF()

for v in range(1, n + 1):
    for i in range(k):
        for j in range(i + 1, k):
            cnf.append([-myvar(v * k + i), -myvar(v * k + j)])
            
for v in range(1, n + 1):
    cnf.append([myvar(v * k + i) for i in range(k)])

for a, b in edges:
    for i in range(k):
        cnf.append([-myvar(a * k + i), -myvar(b * k + i)])

for v in range(1, n + 1):
    for i in range(k):
        cnf.append([-myvar(v * k + i), myvar((n + 1) * k + i)])
        
for i in range(k - 1):
    cnf.append([myvar((n + 1) * k + i), -myvar((n + 1) * k + i + 1)])
    
cnf.append([myvar((n + 1) * k)])

#for v in range(1, n + 1):
#    cnf.extend(CardEnc.equals([myvar(v * k + i) for i in range(k)], 1, vpool=vpool, encoding=EncType.totalizer).clauses)

    
with Minisat22(bootstrap_with=cnf.clauses) as s:
    if s.solve(assumptions=[-myvar((n + 1) * k + i) for i in range(k)][::-1]):
    #if s.solve(): 
        colors = {key // k : key % k for key, value in zip(list(range(k, (n + 1) * k)), s.get_model()) if value > 0}
        print(f"SAT {colors}")
    else:
        print("UNSAT")
        core = s.get_core()
        #print(s.get_core())
        #print([myvar((n + 1) * k + i) for i in range(k)])
        print(f"Core: {[i + 1 for i in range(k) if -myvar((n + 1) * k + i) in core]}")

UNSAT
Core: [2, 3, 4]
